In [1]:
from initial_comparisons import *
from cost_function_comparison import torch_cost_function, torch_estimated_I
import numpy as np
%load_ext line_profiler
%reload_ext initial_comparisons



C:\Users\Bruker\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Params

num_elem = int(1e7)

rng = np.random.default_rng(seed=69)
x = np.abs( rng.normal(size = num_elem) )
y = np.abs( rng.normal(size = num_elem) )

lineprofile = False

In [13]:
# Profiling symbolic diff



if lineprofile:
    %lprun -f symbolic_diff symbolic_diff( symbolic_g, x, y)
else:

    %timeit symbolic_diff(symbolic_g,x,y)



415 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# Profiling AD Torch

if lineprofile:
    %lprun -f AD_torch AD_torch( torch_g, x, y)
else:
    %timeit AD_torch(torch_g,x,y)

399 ms ± 30.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# Profiling AD Autograd

if lineprofile:
    %lprun -f AD_autograd AD_autograd( autograd_g, x, y)
else:
    %timeit AD_autograd(autograd_g,x,y)

457 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# Profiling Numerical
lineprofile= False

if lineprofile:
    %lprun -f numerical_diff numerical_diff(numpy_g, x, y )
else:
    %timeit numerical_diff(numpy_g, x, y )


868 ms ± 9.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# Try again

# Spherical harmonics. 
# Each determined scan point gives a 2D scattering pattern described by 8 sections.
# The intensity of the scattering pattern is estimated by sum over orders of spherical harmonics along the beam path z.

# ALSO, MATLAB STORES (46,34,34,4,1). 
# THIS WAY OF THINKING INCLUDES PROJECTIONS. EACH ORIENTATION WANTS ITS OWN FUNCTION
# HOWEVER, EACH VOXEL HAS ONLY...



projections = torch.ones( size = (1,1,1,8 )) #? # n, y, x, (q,phi) The last parenthesis is just 8 numbers. 
# Num_segments? Fig4: 16 segments where radial integration is performed indicated by white lines and two circles. Only need 8 of them due to symmetry. In addition a q_range. How is this set?


mask_omega = torch.ones( size = (1,1,1) ) # This is 3D in the order: y, x, z. Describes voxels

a_coeffs = torch.ones(size = (1,1,1,1,1) ) # y, x, z, l, m
Y_lms = torch.ones(size = (1,1,1,1,1) )    # y, x, z, l, m

# Hence, estimated intensity, sums over z, l, m. 

estimated_Is = torch.ones( size = (1,1,1,1 ) ) # n, y, x, (q,phi)=8 ; Goes from voxels to projections. Project a 2D scattering image. As determined in cell above. However radial integration into 8 intensity points. 
measured_Is  = torch.ones( size = (1,1,1,1 ) ) # i.e. projections. 
transparency = torch.ones( size = (1,1,1) )


# ISSUE:
# Going from each voxel to all estimated intensity is not so straight forward, as it is conducted by utilising a range of different functions, coordinate systems, and rotations.
# Moreover, it is somewhat unclear to me how one coefficient and one spherical harmonics function shall represent 8 intensity segments. 
# Seems like there is something about retrieving Y_lm coeff as vectors and matrix multiply with powers of cosine.

# Testing purposes, assume one intensity

estimated_Is = torch_estimated_I(a_coeffs, Y_lms)

print(estimated_Is)

print(a_coeffs)
grad = AD_torch_general(torch_cost_function, mask_omega, Y_lms, measured_Is, transparency , a_coeffs= a_coeffs) # args just passed. a_coeffs has to be passed as kwargs. 

print(grad)

tensor([[1.]])
tensor([[[[[1.]]]]])
{'a_coeffs': tensor([[[[[1.]]]]])}
tensor([[[[[[2.]]]]]])


In [2]:
mask_omega = torch.ones( size = (1,1,1) ) # This is 3D in the order: y, x, z. Describes voxels

a_coeffs = torch.ones(size = (1,1,1,1,1) ) *69 # y, x, z, l, m
Y_lms = torch.ones(size = (1,1,1,1,1) )  *169   # y, x, z, l, m

# Hence, estimated intensity, sums over z, l, m. 

estimated_Is = torch.ones( size = (1,1,1,1 ) ) # n, y, x, (q,phi)=8 ; Goes from voxels to projections. Project a 2D scattering image. As determined in cell above. However radial integration into 8 intensity points. 
estimated_Is = torch_estimated_I(a_coeffs, Y_lms)
measured_Is  = torch.ones( size = (1,1,1,1 ) )  # i.e. projections. 
transparency = torch.ones( size = (1,1,1) ) * 0.5


%lprun -f AD_torch_general AD_torch_general( torch_cost_function, mask_omega, Y_lms, measured_Is, transparency , a_coeffs= a_coeffs)

Timer unit: 1e-07 s

Total time: 0.001355 s
File: c:\Users\Bruker\OneDrive\Dokumenter\NTNU\XRD_CT\Autodiff_package\initial_comparisons.py
Function: AD_torch_general at line 86

Line #      Hits         Time  Per Hit   % Time  Line Contents
    86                                           def AD_torch_general(func, *args, **kwargs) -> torch.tensor:
    87                                               '''
    88                                               Applies automatic differentiation provided by pytorch
    89                                           
    90                                               Parameters: 
    91                                               -----------
    92                                               func: function
    93                                                   The cost function to be called
    94                                               *args: list
    95                                                   The arguments of the cost f